# Granite R2 Embeddings Notebook

This notebook demonstrates the capabilities of IBM's Granite R2 embedding models for semantic search and information retrieval. It provides comprehensive examples of using the `granite-embedding-english-r2` retriever and `granite-embedding-reranker-english-r2` reranker models for various tasks.

## Key Features Demonstrated:

- **Basic Embedding Generation**: Simple text encoding and similarity computation
- **Semantic Search Pipeline**: Complete retrieval and reranking workflow
- **Long Document Processing**: Handling documents up to 8,192 tokens
- **Code Search**: Natural language queries for code snippets
- **Table Retrieval**: Searching structured data in markdown tables
- **Batch Processing**: Efficient encoding of large document collections
- **Multi-Turn Conversations**: Contextual search across conversational data

## Models Used:

- **Primary Retriever**: `ibm-granite/granite-embedding-english-r2` (149M parameters, 768-dim embeddings)
- **Reranker**: `ibm-granite/granite-embedding-reranker-english-r2` (149M parameters)

## Architecture:

The notebook implements a two-stage retrieval system:
1. **Fast Retrieval**: Initial candidate selection using dense embeddings
2. **Precise Reranking**: Cross-encoder refinement for improved accuracy

This approach balances speed and accuracy, making it suitable for production environments where both performance and quality matter.


In [110]:
# Load necessary packages
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import numpy as np
from sentence_transformers import util
from torch import Tensor

In [111]:
# Load retriever and reranker
import torch

if torch.cuda.is_available():
    model_kwargs = {"torch_dtype": torch.bfloat16, 'attn_implementation': 'flash_attention_2'}
else:
    model_kwargs = {}

granite_retriever = SentenceTransformer('ibm-granite/granite-embedding-english-r2',
                                        model_kwargs=model_kwargs)
granite_reranker = CrossEncoder('ibm-granite/granite-embedding-reranker-english-r2',
                                model_kwargs=model_kwargs)

In [112]:
# Encode some text
documents = [
    "Granite models are designed for enterprise applications",
    "Information retrieval systems need fast and accurate embeddings",
    "Machine learning models can process natural language"
]

In [113]:
# Encode query and documents
model = granite_retriever

# Compute the query embedding
query = "What's the purpose of the granite models?"
query_embedding = model.encode_query(query)

# Compute the corpus embeddings
doc_embeddings = model.encode_document(documents)

# Compute cosine similarity
similarities = util.cos_sim(query_embedding, doc_embeddings)
print(f"Similarities: {similarities}")

# Get most relevant document
best_idx = np.argmax(similarities)
print(f"Most relevant: {documents[best_idx]}")

Similarities: tensor([[0.8727, 0.7180, 0.7343]])
Most relevant: Granite models are designed for enterprise applications


# Semantic Search with Retrieval and Reranking

This guide demonstrates implementing a two-stage semantic search system using IBM's Granite R2 embeddings. The system combines **fast retrieval** with **optional reranking** for optimal balance between speed and accuracy.

## Architecture Overview

The search pipeline consists of two stages:

1. **Fast Retrieval Stage**: Dense embedding similarity search to quickly identify top-k candidates
2. **Optional Reranking Stage**: Cross-encoder model refines results for improved precision

## Key Components

### Models Used
- **Retriever**: `ibm-granite/granite-embedding-english-r2` (768-dim dense embeddings)
- **Reranker**: `ibm-granite/granite-embedding-reranker-english-r2` (cross-encoder for refinement)

### Core Search Function

The `search()` function implements a two-stage pipeline:
- Encodes queries using the retriever model
- Performs semantic search to get top-k candidates  
- Optionally applies reranking with cross-encoder scoring
- Returns ranked results with confidence scores

The optional reranking stage provides significant accuracy improvements while maintaining reasonable query latency, making this approach suitable for production semantic search applications.


In [114]:
from sentence_transformers import SentenceTransformer, CrossEncoder
import numpy as np

In [115]:
# Your document corpus
corpus = [
    "Python is a high-level programming language",
    "Machine learning models require training data",
    "Natural language processing enables text understanding",
    "Deep learning uses neural networks with multiple layers",
    "Data science combines statistics and programming",
    # ... your documents here
]

In [116]:
# Step 1: Encode corpus once (can be cached)
corpus_embeddings = granite_retriever.encode(corpus, convert_to_tensor=True)

In [142]:
def search(corpus:list[str],
           corpus_embeddings,
           query,
           top_k=20,
           retriever: SentenceTransformer = granite_retriever,
           reranker: CrossEncoder | None = granite_reranker):
    """
    Perform semantic search with optional reranking on a document corpus.

    This function implements a two-stage retrieval pipeline:
    1. Fast retrieval: Uses dense embeddings to quickly identify top-k candidates
    2. Optional reranking: Applies cross-encoder scoring for improved precision

    Args:
        corpus (list[str]): List of document texts to search
        corpus_embeddings (Tensor): Pre-computed embeddings of the document corpus,
            typically obtained from retriever.encode(). Shape: (num_docs, embedding_dim)
        query (str): Natural language search query to find relevant documents
        top_k (int, optional): Number of candidates to retrieve in initial stage.
            Defaults to 20. Higher values increase recall but may impact performance.
        retriever (SentenceTransformer, optional): Sentence transformer model for
            encoding the query. Defaults to granite_retriever.
        reranker (CrossEncoder|None, optional): Cross-encoder model for reranking
            retrieved candidates. If None, only retrieval scores are used.
            Defaults to granite_reranker.

    Returns:
        list[dict]: Top 5 ranked results after reranking (or retrieval if no reranker).
            Each dict contains:
            - 'corpus_id' (int): Index of document in original corpus
            - 'score' (float): Relevance score (reranker score if available, else retrieval score)

    Notes:
        - The function always returns exactly 5 results (or fewer if corpus is smaller)
        - Reranking significantly improves precision but adds latency (~50-100ms per query)
        - Query encoding uses retriever.encode_query() for query-specific optimization
    """
    def get_text(item):
        if isinstance(item, str):
            return item
        elif isinstance(item, dict):
            return item.get('text', '')
        else:
            return ''

    query_embedding = retriever.encode_query(query, convert_to_tensor=True)

    # Find top-k with retriever
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k)[0]

    # Step 3: Rerank with cross-encoder
    if reranker is not None:
        cross_inp = [[query, get_text(corpus[hit['corpus_id']])] for hit in hits]
        cross_scores = reranker.predict(cross_inp)

        # Sort by reranker scores
        for idx, score in enumerate(cross_scores):
            hits[idx]['score'] = score

        hits = sorted(hits, key=lambda x: x['score'], reverse=True)

    return hits[:top_k]  # Return top 5 after reranking

In [118]:
# Use it
# Pardon our reranker, we're working on fixing it...

results = search(corpus, corpus_embeddings, "What is machine learning?", reranker=None)
for hit in results:
    print(f"Score: {hit['score']:.4f} | {corpus[hit['corpus_id']]}")

Score: 0.8633 | Machine learning models require training data
Score: 0.8164 | Data science combines statistics and programming
Score: 0.8086 | Deep learning uses neural networks with multiple layers
Score: 0.7969 | Natural language processing enables text understanding
Score: 0.7773 | Python is a high-level programming language


## Long Document Embedding with Granite R2

This section demonstrates the **Granite R2 embedding model's ability to handle long documents** effectively, showcasing its extended context window of **8,192+ tokens**. This capability makes it particularly suitable for processing comprehensive documents, research papers, articles, and other lengthy text content without requiring chunking strategies.

### Key Features Demonstrated:

- **Extended Context Processing**: Handle documents up to 8,192 tokens in a single pass
- **Semantic Understanding**: Maintain coherent understanding across the entire document
- **Query-Document Similarity**: Compute relevance scores between queries and full documents
- **No Chunking Required**: Process complete documents without splitting strategies

The following example uses a comprehensive Wikipedia article on Waldorf education (~8,000+ tokens) to demonstrate how Granite R2 can encode and query long documents while maintaining semantic accuracy across the entire text.


In [119]:
# The granite embedding r2 blog is the input
long_document="""
Waldorf education, also known as Steiner education, is based on the educational philosophy of Rudolf Steiner, the founder of anthroposophy. Its pedagogy strives to develop pupils\' intellectual, artistic, and practical skills in an integrated and holistic manner. The cultivation of pupils\' imagination and creativity in ways that serve their developmental needs is a strong focus.\n\nIndividual teachers and schools have a great deal of autonomy in determining curriculum content, teaching methodology, and governance. Qualitative assessments of student work are integrated into the daily life of the classroom, with quantitative testing playing a minimal role and standardized testing usually limited to what is required to enter post-secondary education.\n\nThe first Waldorf school opened in 1919 in Stuttgart, Germany. A century later, it has become the largest independent school movement in the world, with more than 1,200 independent schools and nearly 2,000 kindergartens located in 75 countries, as well as more than 500 centers for special education in more than 40 countries. There are also a number of Waldorf-based public schools, charter schools and academies, as well as an active Waldorf homeschooling movement. Germany, the United States and the Netherlands are the countries with the most Waldorf schools.\n\nCritics of Waldorf education like Roger Rawlings point out the sectarian nature of anthroposophy and the incorporation of Steiner\'s esoteric ideas into the educational curriculum. Waldorf schools have also been linked to the outbreak of infectious diseases due to the vaccine hesitancy of many anthroposophists.\n\nOrigins and history\n\nThe first school based upon the ideas of Rudolf Steiner was opened in 1919 in response to a request from Emil Molt, the owner and managing director of the Waldorf-Astoria Cigarette Company in Stuttgart, Germany. This is the source of the name Waldorf, which is now trademarked in some countries when used in connection with the overall method that grew out of this original Waldorf school. Molt\'s proposed school would educate the children of employees of the factory.\n\nMolt was a follower of Anthroposophy, the esoteric spiritual movement based on the notion that an objectively comprehensible spiritual realm exists and can be observed by humans, and a close confidant of Rudolf Steiner, Anthroposophy\'s founder and spiritual leader. Many of Steiner\'s ideas influenced the pedagogy of the original Waldorf school and still play a central role in modern Waldorf classrooms: reincarnation, karma, the existence of spiritual beings (and the idea that children are themselves spiritual beings) and eurythmy.\n\nAs the co-educational school also served children from outside the factory, it included children from a diverse social spectrum. It was also the first comprehensive school, serving children of all genders, abilities, and social classes, in Germany.\n\nWaldorf education became more widely known in Britain in 1922 through lectures Steiner gave at a conference at Oxford University on educational methods. Two years later, on his final trip to Britain at Torquay in 1924, Steiner delivered a Waldorf teacher training course. The first school in England (Michael Hall) was founded in 1925; the first in the United States (the Rudolf Steiner School in New York City) in 1928. By the 1930s, numerous schools inspired by the original and/or Steiner\'s pedagogical principles had opened in Germany, Switzerland, the Netherlands, Norway, Austria, Hungary, the United States, and England.\n\nFrom 1933 to 1945, political interference from the Nazi regime limited and ultimately closed most Waldorf schools in Europe, with the exception of the British, Swiss, and some Dutch schools. The affected schools were reopened after the Second World War, though those in  Soviet-dominated Eastern Germany were closed again a few years later by the Communist German Democratic Republic government.\n\nIn North America, the number of Waldorf schools increased from nine in the United States and one in Canada in 1967 to more than 200 in the United States and over 20 in Canada as of 2021. There are currently 29 Steiner schools in the United Kingdom and three in the Republic of Ireland.\n\nAfter the dissolution of the Soviet Union, Waldorf schools again began to proliferate in Central and Eastern Europe. More recently, many schools have opened in Asia, especially in China. There are currently over 1,200 independent Waldorf schools worldwide.\n\nDevelopmental approach\nThe structure of Waldorf education follows a theory of childhood development devised by Rudolf Steiner, utilizing distinct learning strategies for each of three distinct developmental stages: early childhood, elementary education, and secondary education, each lasting approximately seven years. Aside from their spiritual underpinnings, Steiner\'s seven-year stages are broadly similar to those later described by Jean Piaget. The stated purpose of this approach is to awaken the "physical, behavioral, emotional, cognitive, social, and spiritual" aspects of each individual, fostering creative and inquisitive thought. \n\nSteiner\'s ideas are related to earlier educational theories developed by Comenius and Pestalozzi.\n\nEarly childhood\nWaldorf pedagogical theory considers that during the first years of life children learn best by being immersed in an environment they can learn through un-selfconscious imitation of practical activities. The early childhood curriculum therefore centers on experiential education, allowing children to learn by example, and opportunities for imaginative play. The overall goal of the curriculum is to "imbue the child with a sense that the world is good".\n\nWaldorf preschools employ a regular daily routine that includes free play, artistic work (e.g. drawing, painting or modeling), circle time (songs, games, and stories), and practical tasks (e.g. cooking, cleaning, and gardening), with rhythmic variations. Periods of outdoor recess are also usually included. The classroom is intended to resemble a home, with tools and toys usually sourced from simple, natural materials that lend themselves to imaginative play. The use of natural materials has been widely praised as fulfilling children\'s aesthetic needs, encouraging their imagination, and reinforcing their identification with nature, though one pair of reviewers questioned whether the preference for natural, non-manufactured materials is "a reaction against the dehumanizing aspects of nineteenth-century industrialization" rather than a "reasoned assessment of twenty-first century children\'s needs".\n\nPre-school and kindergarten programs generally include seasonal festivals drawn from a variety of traditions, with attention placed on the traditions brought forth from the community. Waldorf schools in the Western Hemisphere have traditionally celebrated Christian festivals, though one sources states that some North American Waldorf schools also include some Jewish holidays.\n\nWaldorf kindergarten and lower grades generally discourage pupils\' use of electronic media such as television and computers. There are a variety of reasons for this: Waldorf educators believe that use of these conflicts with young children\'s developmental needs, media users may be physically inactive, and media may be seen to contain inappropriate or undesirable content and to hamper the imagination.\n\nElementary education\n\nWaldorf pedagogues consider that readiness for formal learning depends upon increased independence of character, temperament, habits, and memory, one of the markers of which is the loss of the baby teeth. Formal instruction in reading, writing, and other academic disciplines are therefore not introduced until students enter the elementary school, when pupils are around seven years of age. Steiner believed that engaging young children in abstract intellectual activity too early would adversely affect their growth and development.\n\nWaldorf elementary schools (ages 7–14) emphasize cultivating children\'s emotional life and imagination. In order that students can connect more deeply with the subject matter, academic instruction is presented through artistic work that includes story-telling, visual arts, drama, movement, vocal and instrumental music, and crafts. The core curriculum includes language arts, mythology, history, geography, geology, algebra, geometry, mineralogy, biology, astronomy, physics, chemistry, and nutrition. The school day generally begins with a one-and-a-half to two-hour, cognitively oriented academic lesson, or "Main lesson", that focuses on a single theme over the course of about a month\'s time. This typically begins with introductory activities that may include singing, instrumental music, and recitations of poetry, generally including a verse written by Steiner for the start of a school day.\n\nElementary school educators\' stated task is to present a role model children will naturally want to follow, gaining authority through fostering rapport and "nurturing curiosity, imagination, and creativity". The declared goal of this second stage is to "imbue children with a sense that the world is beautiful". There is little reliance on standardized textbooks.\n\nWaldorf elementary education allows for individual variations in the pace of learning, based upon the expectation that a child will grasp a concept or achieve a skill when he or she is ready. Cooperation takes priority over competition. This approach also extends to physical education; competitive team sports are introduced in upper grades.\n\nEach class normally remains together as a cohort throughout their years, developing as a quasi-familial social group whose members know each other quite deeply. In the elementary years, a core teacher teaches the primary academic subjects. A central role of this class teacher is to provide supportive role models both through personal example and through stories drawn from a variety of cultures, educating by exercising creative, loving authority. Class teachers are normally expected to teach a group of children for several years, a practice known as looping. The traditional goal was for the teacher to remain with a class for the eight years of the "lower school" cycle, but in recent years the duration of these cycles has been increasingly treated flexibly. Already in first grade, specialized teachers teach many of the subjects, including music, crafts, movement, and two foreign languages from complementary language families (in English-speaking countries often German and either Spanish or French); these subjects remain central to the curriculum throughout the elementary school years.\n\nWhile class teachers serve a valuable role as personal mentors, establishing "lasting relationships with pupils", especially in the early years, Ullrich documented problems when the same class teacher continues into the middle school years. Noting that there is a danger of any authority figure limiting students enthusiasm for inquiry and assertion of autonomy, he emphasized the need for teachers to encourage independent thought and explanatory discussion in these years, and cited approvingly a number of schools where the class teacher accompanies the class for six years, after which specialist teachers play a significantly greater role.\n\nFour temperaments\nSteiner considered children\'s cognitive, emotional and behavioral development to be interlinked. When students in a Waldorf school are grouped, it is generally not by a singular focus on their academic abilities. Instead Steiner adapted the proto-psychological concept of the classic four temperaments – melancholic, sanguine, phlegmatic and choleric – for pedagogical use in the elementary years. Steiner indicated that teaching should be differentiated to accommodate the different needs that these psychophysical types represent. For example, "cholerics are risk takers, phlegmatics take things calmly, melancholics are sensitive or introverted, and sanguines take things lightly". Today Waldorf teachers may work with the notion of temperaments to differentiate their instruction. Seating arrangements and class activities may be planned taking into account the temperaments of the students but this is often not readily apparent to observers. Steiner also believed that teachers must consider their own temperament and be prepared to work with it positively in the classroom, that temperament is emergent in children, and that most people express a combination of temperaments rather than a pure single type.\n\nSecondary education\nIn most Waldorf schools, pupils enter secondary education when they are about fourteen years old. Secondary education is provided by specialist teachers for each subject. The education focuses much more strongly on academic subjects, though students normally continue to take courses in art, music, and crafts. The curriculum is structured to foster pupils\' intellectual understanding, independent judgment, and ethical ideals such as social responsibility, aiming to meet the developing capacity for abstract thought and conceptual judgment.\n\nIn the third developmental stage (14 years old and up), children in Waldorf programs are supposed to learn through their own thinking and judgment. Students are asked to understand abstract material and expected to have sufficient foundation and maturity to form conclusions using their own judgment.  The intention of the third stage is to "imbue children with a sense that the world is true".\n\nThe overarching goals are to provide young people the basis on which to develop into free, morally responsible, and integrated individuals, with the aim of helping young people "go out into the world as free, independent and creative beings". No independent studies have been published as to whether or not Waldorf education achieves this aim.\n\nEducational theory and practice\n\nThe philosophical foundation of the Waldorf approach, anthroposophy, underpins its primary pedagogical goals: to provide an education that enables children to become free human beings, and to help children to incarnate their "unfolding spiritual identity", carried from the preceding spiritual existence, as beings of body, soul, and spirit in this lifetime. Educational researcher Martin Ashley suggests that the latter role would be problematic for secular teachers and parents in state schools, and the commitment to a spiritual background both of the child and the education has been problematic for some committed to a secular perspective.\n\nWhile anthroposophy underpins the curriculum design, pedagogical approach, and organizational structure, it is explicitly not taught within the school curriculum and studies have shown that Waldorf pupils have little awareness of it. Tensions may arise within the Waldorf community between the commitment to Steiner\'s original intentions, which has sometimes acted as a valuable anchor against following educational fads, and openness to new directions in education, such as the incorporation of new technologies or modern methods of accountability and assessment.\n\nWaldorf schools frequently have striking architecture, employing walls meeting at varied angles (not only perpendicularly) to achieve a more fluid, less boxed-in feeling to the space. The walls are often painted in subtle colors, often with a lazure technique, and include textured surfaces.\n\nAssessment\nThe schools primarily assess students through reports on individual academic progress and personal development. The emphasis is on characterization through qualitative description. Pupils\' progress is primarily evaluated through portfolio work in academic blocks and discussion of pupils in teacher conferences. Standardized tests are rare, with the exception of examinations necessary for college entry taken during the secondary school years. Letter grades are generally not given until students enter high school at 14–15 years, as the educational emphasis is on children\'s holistic development, not solely their academic progress. Pupils are not normally asked to repeat years of elementary or secondary education. It is noted that Waldorf education is not a matter of "assessment-driven instruction or vice-versa" and there is no anxiety-producing experience on the part of the learner of suddenly being tested.\n\nCurriculum\n\nThough Waldorf schools are autonomous institutions not required to follow a prescribed curriculum (beyond those required by local governments) there are widely agreed upon guidelines for the Waldorf curriculum, supported by the schools\' common principles. The schools offer a wide curriculum "governed by close observation and recording of what content motivates children at different ages" and including within it, for example, the English, Welsh and Northern Irish National Curriculum.\n\nThe main academic subjects are introduced through up to two-hour morning lesson blocks that last for several weeks.  These lesson blocks are horizontally integrated at each grade level in that the topic of the block will be infused into many of the activities of the classroom and vertically integrated in that each subject will be revisited over the course of the education with increasing complexity as students develop their skills, reasoning capacities and individual sense of self. This has been described as a spiral curriculum.\n\nMany subjects and skills not considered core parts of mainstream schools, such as art, music, gardening, and mythology, are central to Waldorf education. Students learn a variety of fine and practical arts. Elementary students paint, draw, sculpt, knit, weave, and crochet. Older students build on these experiences and learn new skills such as pattern-making and sewing, wood and stone carving, metal work, book-binding, and doll or puppet making. Fine art instruction includes form drawing, sketching, sculpting, perspective drawing and other techniques.\n\nMusic instruction begins with singing in early childhood and choral instruction remains an important component through the end of high school. Pupils usually learn to play pentatonic flutes, recorders and/or lyres in the early elementary grades. Around age 9, diatonic recorders and orchestral instruments are introduced.\n\nCertain subjects are largely unique to the Waldorf schools. Foremost among these is eurythmy, a movement art usually accompanying spoken texts or music which includes elements of drama and dance and is designed to provide individuals and classes with a "sense of integration and harmony". Although found in other educational contexts, cooking, farming, and environmental and outdoor education have long been incorporated into the Waldorf curriculum. Other differences include: non-competitive games and free play in the younger years as opposed to athletics instruction; instruction in two foreign languages from the beginning of elementary school; and an experiential-phenomenological approach to science whereby students observe and depict scientific concepts in their own words and drawings rather than encountering the ideas first through a textbook.\n\nThe Waldorf curriculum aims to incorporate multiple intelligences.\n\nScience\n\nThe scientific methodology of modern Waldorf schools utilizes a so-called "phenomenological approach" to science education employing a methodology of inquiry-based learning aiming to "strengthen the interest and ability to observe" in pupils. The main aim is to cultivate a sense of "wholeness" in regard to the human\'s place in nature.\n\nOther experts have called into question the quality of this phenomenological approach if it fails to educate Waldorf students on basic tenets of scientific fact. One study conducted by California State University at Sacramento researchers outlined numerous theories and ideas prevalent throughout Waldorf curricula that were patently pseudoscientific and steeped in magical thinking. These included the idea that animals evolved from humans, that human spirits are physically incarnated into "soul qualities that manifested themselves into various animal forms", that the current geological formations on Earth have evolved through so-called "Lemurian" and "Atlantiean" epochs, and that the four kingdoms of nature are "mineral, plant, animal, and man". All of these are directly contradicted by mainstream scientific knowledge and have no basis in any form of conventional scientific study. Contradictory notions found in Waldorf textbooks are distinct from factual inaccuracies occasionally found in modern public school textbooks, as the inaccuracies in the latter are of a specific and minute nature that results from the progress of science. The inaccuracies present in Waldorf textbooks, however, are the result of a mode of thinking that has no valid basis in reason or logic. This unscientific foundation has been blamed for the scarcity of systematic empirical research on Waldorf education as academic researchers hesitate in getting involved in studies of Waldorf schools lest it hamper their future career.\n\nOne study of the science curriculum compared a group of American Waldorf school students to American public school students on three different test variables. Two tests measured verbal and non-verbal logical reasoning and the third was an international TIMSS test. The TIMSS test covered scientific understanding of magnetism. The researchers found that Waldorf school students scored higher than both the public school students and the national average on the TIMSS test while scoring the same as the public school students on the logical reasoning tests. However, when the logical reasoning tests measured students\' understanding of part-to-whole relations, the Waldorf students also outperformed the public school students. The authors of the study noted the Waldorf students\' enthusiasm for science, but viewed the science curriculum as "somewhat old-fashioned and out of date, as well as including some doubtful scientific material". Educational researchers Phillip and Glenys Woods, who reviewed this study, criticized the authors\' implication of an "unresolved conflict": that it is possible for supposedly inaccurate science to lead to demonstrably better scientific understanding.\n\nIn 2008, Stockholm University terminated its Waldorf teacher training courses. In a statement the university said "the courses did not encompass sufficient subject theory and a large part of the subject theory that is included is not founded on any scientific base". The dean, Stefan Nordlund, stated "the syllabus contains literature which conveys scientific inaccuracies that are worse than woolly; they are downright dangerous".\n\nInformation technology\nBecause they view human interaction as the essential basis for younger children\'s learning and growth, Waldorf schools view computer technology as being first useful to children in the early teen years, after they have mastered "fundamental, time-honoured ways of discovering information and learning, such as practical experiments and books".\n\nIn the United Kingdom, Waldorf schools are granted an exemption by the Department for Education (DfE) from the requirement to teach ICT as part of Foundation Stage education (ages 3–5). Education researchers John Siraj-Blatchford and David Whitebread praised the DfE for making this exemption, highlighting Waldorf education\'s emphasis on simplicity of resources and the way the education cultivates the imagination.\n\nWaldorf schools have been popular with some parents working in the technology sector in the United States, including those from some of the most advanced technology firms. A number of technologically oriented parents from one school expressed their conviction that younger students do not need the exposure to computers and technology, but benefit from creative aspects of the education; one Google executive was quoted as saying "I fundamentally reject the notion you need technology aids in grammar school."\n\nSpirituality\nWaldorf education aims to educate children about a wide range of religious traditions without favoring any one of these. One of Steiner\'s primary aims was to establish a spiritual yet nondenominational setting for children from all backgrounds that recognized the value of role models drawn from a wide range of literary and historical traditions in developing children\'s fantasy and moral imaginations. Indeed, for Steiner, education was an activity which fosters the human being\'s connection to the divine and is thus inherently religious.\n\nWaldorf schools were historically "Christian based and theistically oriented", as they expand into different cultural settings they are adapting to "a truly pluralistic spirituality". Waldorf theories and practices are often modified from their European and Christian roots to meet the historical and cultural traditions of the local community. Examples of such adaptation include the Waldorf schools in Israel and Japan, which celebrate festivals drawn from these cultures, and classes in the Milwaukee Urban Waldorf school, which have adopted African American and Native American traditions. Such festivals, as well as assemblies generally, which play an important role in Waldorf schools, generally center on classes presenting their work.\n\nReligion classes are usually absent from United States Waldorf schools,  are a mandatory offering in some German federal states, whereby in Waldorf schools each religious denomination provides its own teachers for the classes, and a non-denominational religion class is also offered. In the United Kingdom, public Waldorf schools are not categorized as "Faith schools".\n\nTom Stehlik places Waldorf education in a humanistic tradition, and contrasts its philosophically grounded approach to "value-neutral" secular state schooling systems.\n\nTeacher education\n\nWaldorf teacher education programs offer courses in child development, the methodology of Waldorf teaching, academic subjects appropriate to the future teachers\' chosen specialty, and the study of pedagogical texts and other works by Steiner. For early childhood and elementary school teachers, the training includes considerable artistic work in storytelling, movement, painting, music, and handwork.\n\nWaldorf teacher education includes social–emotional development as "an integral and central element", which is unusual for teacher trainings. A 2010 study found that students in advanced years of Waldorf teacher training courses scored significantly higher than students in non-Waldorf teacher trainings on three measures of empathy: perspective taking, empathic concern and fantasy.\n\nGovernance\n\nIndependent schools\nOne of Waldorf education\'s central premises is that all educational and cultural institutions should be self-governing and should grant teachers a high degree of creative autonomy within the school; this is based upon the conviction that a holistic approach to education aiming at the development of free individuals can only be successful when based on a school form that expresses these same principles. Most Waldorf schools are not directed by a principal or head teacher, but rather by a number of groups, including:\n The college of teachers, who decide on pedagogical issues, normally on the basis of consensus. This group is usually open to full-time teachers who have been with the school for a prescribed period of time. Each school is accordingly unique in its approach, as it may act solely on the basis of the decisions of the college of teachers to set policy or other actions pertaining to the school and its students.\n The board of trustees, who decide on governance issues, especially those relating to school finances and legal issues, including formulating strategic plans and central policies.\n\nParents are encouraged to take an active part in non-curricular aspects of school life. Waldorf schools have been found to create effective adult learning communities.\n\nThere are coordinating bodies for Waldorf education at both the national (e.g. the Association of Waldorf Schools of North America and the Steiner Waldorf Schools Fellowship in the UK and Ireland) and international level (e.g. International Association for Waldorf Education and The European Council for Steiner Waldorf Education (ECSWE)). These organizations certify the use of the registered names "Waldorf" and "Steiner school" and offer accreditations, often in conjunction with regional independent school associations.\n\nState-funded schools\nIndependent schools receive complete or partial funding in much of Europe, particularly in Northern and Eastern Europe. Sweden, Finland, Holland, and Slovakia provide over 90% of independent schools\' funding, while Slovenia, Germany, Belgium, Luxembourg, Ireland, Hungary, Estonia, the Czech Republic, Denmark, Spain, and Portugal provide the majority of independent schools\' funding.  In countries outside of this region, funding for independent schools varies widely.\n\nUnited States\n\nThe first US Waldorf-inspired public school, the Yuba River Charter School in California, opened in 1994. The Waldorf public school movement is currently expanding rapidly; while in 2010, there were twelve Waldorf-inspired public schools in the United States, by 2018 there were 53 such schools.\n\nMost Waldorf-inspired schools in the United States are elementary schools established as either magnet or charter schools. The first Waldorf-inspired high school was launched in 2008 with assistance from the Bill and Melinda Gates Foundation. While these schools follow a similar developmental approach as the independent schools, Waldorf-inspired schools must demonstrate achievement on standardized tests in order to continue receiving public funding. Studies of standardized test scores suggest that students at Waldorf-inspired schools tend to score below their peers in the earliest grades and catch up or surpass their peers by middle school. One study found that students at Waldorf-inspired schools watch less television and spend more time engaging in creative activities or spending time with friends. Public Waldorf schools\' need to demonstrate achievement through standardized test scores has encouraged increased use of textbooks and expanded instructional time for academic subjects.\n\nA legal challenge alleging that California school districts\' Waldorf-inspired schools violated the First and Fourteenth Amendments of the United States Constitution and Article IX of the California Constitution was dismissed on its merits in 2005 and on appeal in 2007 and 2012. Further, some Waldorf schools in English-speaking countries have met opposition due to vaccine hesitancy among the parents of some Waldorf pupils, differences in education standards, and the mystical and antiquated nature of some of Steiner\'s theories. In a 2011 article, Waldorf schools were identified as a risk factor for noncompliance of Measles vaccination.\n\nUnited Kingdom\nThe first state-funded Steiner-Waldorf school in the United Kingdom, the Steiner Academy Hereford, opened in 2008. Since then, Steiner academies have opened in Frome, Exeter and Bristol as part of the government-funded free schools programme.\n\nIn December 2018, Ofsted judged the Steiner Academy Exeter as inadequate and ordered it to be transferred to a multi-academy trust; it was temporarily closed in October 2018 because of concerns. The concerns included significant lapses in safeguarding, and mistreating children with special educational needs and disabilities, and misspending the funding designated for them. Another incident was that in July 2018, two 6-year-old children were found by police having walked out of the school unnoticed, and their parents were not informed until the end of the day. Subsequently, the Steiner Academies in Bristol and Frome have also been judged inadequate by Ofsted, because of concerns over safeguarding and bullying, and a number of private Steiner schools have also been judged inadequate. Overall, several Waldorf schools in the UK have closed in the last decade due to their administrations\' failure to adhere to state-mandated standards of education (e.g. required levels of literacy, safety standards for child welfare, and mistreatment of special needs children).\n\nIn November 2012, BBC News broadcast an item about accusations that the establishment of a state-funded Waldorf School in Frome was a misguided use of public money. The broadcast reported that concerns were being raised about Rudolf Steiner\'s beliefs, stating he "believed in reincarnation and said it was related to race, with black (schwarz) people being the least spiritually developed, and white (weiß) people the most". In 2007 the European Council for Steiner Waldorf Education (ECSWE) issued a statement, "Waldorf schools against discrimination", which said in part, "Waldorf schools do not select, stratify or discriminate amongst their pupils, but consider all human beings to be free and equal in dignity and rights, independent of ethnicity, national or social origin, gender, language, religion, and political or other convictions. Anthroposophy, upon which Waldorf education is founded, stands firmly against all forms of racism and nationalism."\n\nThe British Humanist Association criticised a reference book used to train teachers in Steiner academies for suggesting that the heart is sensitive to emotions and promoting homeopathy, while claiming that Darwinism is "rooted in reductionist thinking and Victorian ethics". Edzard Ernst, emeritus professor of complementary medicine, said that Waldorf schools "seem to have an anti-science agenda". A United Kingdom Department for Education spokeswoman said "no state school is allowed to teach homeopathy as scientific fact" and that free schools "must demonstrate that they will provide a broad and balanced curriculum".\n\nAustralia, New Zealand, and Canada\nAustralia has "Steiner streams" incorporated into a small number of existing government schools in some states; in addition, independent Steiner-Waldorf schools receive partial government funding. The majority of Steiner-Waldorf schools in New Zealand are Integrated Private Schools under The Private Schools Integration Act 1975, thus receiving full state funding. In the Canadian provinces of British Columbia, Quebec and Alberta, all private schools receive partial state funding.\n\nRussia\nThe first Steiner school in Russia was established in 1992 in Moscow. That school is now an award-winning government-funded school with over 650 students offering classes for kindergarten and years 1 to 11 (the Russian education system is an eleven year system). There are 18 Waldorf schools in Russia and 30 kindergartens. Some are government funded (with no fees) and some are privately funded (with fees for students). As well as five Waldorf schools in Moscow, there are also Waldorf schools in Saint Petersburg, Irkutsk, Jaroslawl, Kaluga, Samara, Zhukovskiy, Smolensk, Tomsk, Ufa, Vladimir, Voronezh, and Zelenograd. The Association of Russian Waldorf Schools was founded in 1995 and now has 21 members.\n\nHomeschooling\nWaldorf-inspired home schools typically obtain their program information through informal parent groups, online, or by purchasing a curriculum. Waldorf homeschooling groups are not affiliated with the Association of Waldorf Schools of North America (AWSNA), which represents independent schools and it is unknown how many home schools use a Waldorf-inspired curriculum.\n\nEducationalist Sandra Chistolini suggests that parents offer their children Waldorf-inspired homeschooling because "the frustration and boredom some children feel in school are eliminated and replaced with constant attention to the needs of childhood [and] connections between content and the real world".\n\nSocial engagement\nSteiner\'s belief that all people are imbued with a spiritual core has fueled Waldorf schools\' social mission. The schools have always been coeducational and open to children of all social classes. They were designed from the beginning to be comprehensive, 12-year schools under the direction of their own teachers, rather than the state or other external authorities, all radical principles when Steiner first articulated them.\n\nSocial renewal and transformation remain primary goals for Waldorf schools, which seek to cultivate pupils\' sense of social responsibility. Studies suggest that this is successful; Waldorf pupils have been found to be more interested in and engaged with social and moral questions and to have more positive attitudes than students from mainstream schools, demonstrating activism and self-confidence and feeling empowered to forge their own futures.\n\nWaldorf schools build close learning communities, founded on the shared values of its members, in ways that can lead to transformative learning experiences that allow all participants, including parents, to become more aware of their own individual path, but which at times also risk becoming exclusive. Reports from small-scale studies suggest that there are lower levels of harassment and bullying in Waldorf schools and that European Waldorf students have much lower rates of xenophobia and gender stereotypes than students in any other type of schools. Betty Reardon, a professor and peace researcher, gives Waldorf schools as an example of schools that follow a philosophy based on peace and tolerance.\n\nMany private Waldorf schools experience a tension between these social goals and the way tuition fees act as a barrier to access to the education by less well-off families. Schools have attempted to improve access for a wider range of income groups by charging lower fees than comparable independent schools, by offering a sliding scale of fees, and/or by seeking state support.\n\nIntercultural links in socially polarized communities\n\nWaldorf schools have linked polarized communities in a variety of settings.\n Under the apartheid regime in South Africa, the Waldorf school was one of the few schools in which children of all apartheid racial classifications attended the same classes, despite the ensuing loss of state aid. A Waldorf training college in Cape Town, the Novalis Institute, was referenced during UNESCO\'s Year of Tolerance for being an organization that was working towards reconciliation in South Africa.\n The first Waldorf school in West Africa was founded in Sierra Leone to educate boys and girls orphaned by the country\'s civil war. The school building is a passive solar building built by the local community, including the students.\n In Israel, the Harduf Kibbutz Waldorf school includes both Jewish and Arab faculty and students and has extensive contact with the surrounding Arab communities. It also runs an Arab-language Waldorf teacher training. A joint Arab-Jewish Waldorf kindergarten, Ein Bustan, was founded in Hilf (near Haifa) in 2005 while an Arabic language multi-cultural Druze/Christian/Muslim Waldorf school has operated in Shefa-\'Amr since 2003. In Lod, a teacher training program brings together Israeli Arabs and Jews on an equal basis, with the goals of improving Arab education in Israel and offering new career paths to Arab women.\n\n In Brazil, a Waldorf teacher, Ute Craemer, founded Associação Comunitária Monte Azul, a community service organization providing childcare, vocational training and work, social services including health care, and Waldorf education to more than 1,000 residents of poverty-stricken areas (Favelas) of São Paulo.\n In Nepal, the Tashi Waldorf School in the outskirts of Kathmandu teaches mainly disadvantaged children from a wide variety of cultural backgrounds. It was founded in 1999 and is run by Nepalese staff. In addition, in the southwest Kathmandu Valley a foundation provides underprivileged, disabled and poor adults with work on a biodynamic farm and provides a Waldorf school for their children.\n The T.E. Mathews Community School in Yuba County, California, serves high-risk juvenile offenders, many of whom have learning disabilities. The school switched to Waldorf methods in the 1990s. A 1999 study of the school found that students had "improved attitudes toward learning, better social interaction and excellent academic progress". This study identified the integration of the arts "into every curriculum unit and almost every classroom activity" as the most effective tool to help students overcome patterns of failure. The study also found significant improvements in reading and math scores, student participation, focus, openness and enthusiasm, as well as emotional stability, civility of interaction and tenacity.\n\nWaldorf education also has links with UNESCO. In 2008, 24 Waldorf schools in 15 countries were members of the UNESCO Associated Schools Project Network. The Friends of Waldorf Education is an organization whose purpose is to support, finance and advise the Waldorf movement worldwide, particularly in disadvantaged settings.\n\nReception\n\nEvaluations of students\' progress\n\nAlthough studies about Waldorf education tend to be small-scale and vary in national context, a 2005 independent comprehensive review of the literature concluded there was evidence that Waldorf education encourages academic achievement as well as "creative, social and other capabilities important to the holistic growth of a person".\n\nIn comparison to state school pupils, European Waldorf students are significantly more enthusiastic about learning, report having more fun and being less bored in school, view their school environments as pleasant and supportive places where they are able to discover their personal academic strengths, and have more positive views of the future. Twice as many European Waldorf students as state school pupils report having good relationships with teachers; they also report significantly fewer ailments such as headaches, stomach aches, and disrupted sleep.\n\nA 2007 German study found that an above-average number of Waldorf students become teachers, doctors, engineers, scholars of the humanities, and scientists. Studies of Waldorf students\' artistic capacities found that they averaged higher scores on the Torrance Test of Creative Thinking Ability, drew more accurate, detailed, and imaginative drawings, and were able to develop richer images than comparison groups.\n\nSome observers have noted that Waldorf educators tend to be more concerned to address the needs of weaker students who need support than they are to meet the needs of talented students who could benefit from advanced work.\n\nEducational scholars\n\nProfessor of educational psychology Clifford Mayes said "Waldorf students learn in sequences and paces that are developmentally appropriate, aesthetically stimulating, emotionally supportive, and ecologically sensitive." Professors of education Timothy Leonard and Peter Willis stated that Waldorf education "cultivates the imagination of the young to provide them a firm emotional foundation upon which to build a sound intellectual life".\n\nProfessor of education Bruce Uhrmacher considers Steiner\'s view on education worthy of investigation for those seeking to improve public schooling, saying the approach serves as a reminder that "holistic education is rooted in a cosmology that posits a fundamental unity to the universe and as such ought to take into account interconnections among the purpose of schooling, the nature of the growing child, and the relationships between the human being and the universe at large", and that a curriculum need not be technocratic, but may equally well be arts-based.\n\nThomas Nielsen, assistant professor at the University of Canberra\'s education department, said that imaginative teaching approaches used in Waldorf education (drama, exploration, storytelling, routine, arts, discussion and empathy) are effective stimulators of spiritual-aesthetic, intellectual and physical development, expanding "the concept of holistic and imaginative education" and recommends these to mainstream educators.\n\nAndreas Schleicher, international coordinator of the PISA studies, commented on what he saw as the "high degree of congruence between what the world demands of people, and what Waldorf schools develop in their pupils", placing a high value on creatively and productively applying knowledge to new realms. This enables "deep learning" that goes beyond studying for the next test. Deborah Meier, principal of Mission Hill School and MacArthur grant recipient, while having some "quibbles" about the Waldorf schools, stated: "The adults I know who have come out of Waldorf schools are extraordinary people. That education leaves a strong mark of thoroughness, carefulness, and thoughtfulness."\n\nRobert Peterkin, director of the Urban Superintendents Program at Harvard\'s Graduate School of Education and former Superintendent of Milwaukee Public Schools during a period when Milwaukee funded a public Waldorf school, considers Waldorf education a "healing education" whose underlying principles are appropriate for educating all children.\n\nWaldorf education has also been studied as an example of educational neuroscience ideas in practice.\n\nGermany\nIn 2000, educational scholar Heiner Ullrich wrote that intensive study of Steiner\'s pedagogy had been in progress in educational circles in Germany since about 1990 and that positions were "highly controversial: they range from enthusiastic support to destructive criticism". In 2008, the same scholar wrote that Waldorf schools have "not stirred comparable discussion or controversy... those interested in the Waldorf School today... generally tend to view this school form first and foremost as a representative of internationally recognized models of applied classic reform pedagogy" and that critics tend to focus on what they see as Steiner\'s "occult neo-mythology of education" and to fear the risks of indoctrination in a worldview school, but lose an "unprejudiced view of the varied practice of the Steiner schools". Ullrich himself considers that the schools successfully foster dedication, openness, and a love for other human beings, for nature, and for the inanimate world.\n\nProfessor of Comparative Education Hermann Röhrs describes Waldorf education as embodying original pedagogical ideas and presenting exemplary organizational capabilities.\n\nRelationship with mainstream education\n\nA UK Department for Education and Skills report suggested that Waldorf and state schools could learn from each other\'s strengths: in particular, that state schools could benefit from Waldorf education\'s early introduction and approach to modern foreign languages; combination of block (class) and subject teaching for younger children; development of speaking and listening through an emphasis on oral work; good pacing of lessons through an emphasis on rhythm; emphasis on child development guiding the curriculum and examinations; approach to art and creativity; attention given to teachers\' reflective activity and heightened awareness (in collective child study for example); and collegial structure of leadership and management, including collegial study. Aspects of mainstream practice which could inform good practice in Waldorf schools included: management skills and ways of improving organizational and administrative efficiency; classroom management; work with secondary-school age children; and assessment and record keeping.\n\nAmerican state and private schools are drawing on Waldorf education"less in whole than in part"in expanding numbers. Professor of Education Elliot Eisner sees Waldorf education exemplifying embodied learning and fostering a more balanced educational approach than American public schools achieve. Ernest Boyer, former president of the Carnegie Foundation for the Advancement of Teaching commended the significant role the arts play throughout Waldorf education as a model for other schools to follow. Waldorf schools have been described as establishing "genuine community" and contrasted to mainstream schools, which have been described as "residential areas partitioned by bureaucratic authorities for educational purposes".\n\nMany elements of Waldorf pedagogy have been used in all Finnish schools for many years.\n\nAshley described seven principal ways Waldorf education differed from mainstream approaches: its method of working from the whole to the parts, its attentiveness to child development, its goal of freedom, the deep relationships of teachers to students, the emphasis on experiencing oral traditions, the role of ritual and routine (e.g. welcoming students with a handshake, the use of opening and closing verses, and yearly festivals), the role arts and creativity play, and the Goetheanistic approach to science.\n\nPublic health\n\nVaccine beliefs\nIn US states where nonmedical vaccine exemption is legal, 2015 reports showed Waldorf schools as having a high rate of vaccine exemption within their student populations, however, recent research has shown that in US state schools, child immunization rates often fall below the 95-percent threshold that the Centers for Disease Control say is necessary to provide herd immunity for a community. A 2010 report by the UK Government said that Steiner schools should be considered "high risk populations" and "unvaccinated communities" with respect to children\'s risks of catching measles and contributing to outbreaks. On 19 November 2018, the BBC reported there was an outbreak of chickenpox affecting 36 students at the Asheville Waldorf School located in North Carolina. Out of 152 students at the school, 110 had not received the Varicella vaccine that protects against chickenpox. The United States Advisory Committee on Immunization, the Centers for Disease Control, and the North Carolina Department of Health and Human Services all recommend that all healthy children 12 months of age and older get vaccinated against Varicella. The Guardian reported that several Waldorf schools in California had some of the lowest vaccination rates among kindergarten pupils in the 2017–18 school year, with only 7% of pupils having been vaccinated in one school. In the same article, however, The Guardian also reported that, in a 2019 statement, the International Center for Anthroposophic Medicine and the International Federation of Anthroposophic Medical Associations stressed that anthroposophic medicine, the form of medicine Steiner founded, "fully appreciates the contributions of vaccines to global health and firmly supports vaccinations as an important measure to prevent life threatening diseases".\n\nRudolf Steiner founded the first Waldorf school several years before vaccinations for tetanus, diphtheria, and whooping cough were invented. After such vaccinations became widespread in Europe, Steiner opposed their use in several contexts, writing that vaccination could "impede spiritual development" and lead to a loss of "any urge for a spiritual life". Steiner also thought that these effects would carry over into subsequent reincarnations of the vaccinated person.\n\nThe Association of Waldorf Schools of North America released the following in a statement in 2019:\n\nThe Association of Waldorf Schools of North America wishes to state unequivocally that our educational objectives do not include avoidance of, or resistance to, childhood immunization. The health, safety, and wellbeing of children are our forefront concerns.\n\n All members of our association are schools or institutions that are free to make independent school policy decisions in accordance with AWSNA\'s membership and accreditation criteria. Our membership and accreditation criteria require schools to be compliant with national, state, provincial, and local laws. While policy decisions regarding immunizations may vary from school to school, such decisions are made in accordance with legal requirements set by local, state, provincial or federal government. \n The Association encourages parents to consider their civic responsibility in regards to the decision of whether or not to immunize against any communicable disease, but ultimately, the decision to immunize or not is one made by parents in consultation with their family physician.\n\nIn 2021, Waldorf schools in Germany were been associated with outbreaks of COVID-19 during a pandemic of the disease, as well as reticence to incorporate public health measures relating to disease outbreak.\n\nRace \n\nThe Association of Waldorf Schools of North America (AWSNA) and European Council for Steiner Waldorf Education have put out statements stating that "racist or discriminatory tendencies are not tolerated in Waldorf schools or Waldorf teacher training institutes. The Waldorf school movement explicitly rejects any attempt to misappropriate Waldorf pedagogy or Rudolf Steiner\'s work for racist or nationalistic purposes." Similar statements were put out by the Waldorf school association in Britain ("Our schools do not tolerate racism. Racist views do not accord with Steiner\'s longer term vision of a society in which such distinctions would be entirely irrelevant & modern Steiner Waldorf schools deplore all forms of intolerance, aiming to educate in a spirit of respect & to encourage open-hearted regard for others among the children they educate") and Germany.\n\nThese statements are the necessary response to Rudolf Steiner\'s contradictory beliefs about race: he emphasized the core spiritual unity of all the world\'s peoples, sharply criticized racial prejudice, and articulated beliefs that the individual nature of any person stands higher than any racial, ethnic, national or religious affiliation, yet he asserted a hierarchy of races, with the white race at the top, and associated intelligence with having blonde hair and blue eyes.\n\nIn 2019 a school in Christchurch, New Zealand began considering removing "Rudolf Steiner" from the name of the school "so that the our best ideals are not burdened by historical, philosophical untruths." In 2014, after an investigation by the NZ Ministry of Education, a small school on the Kapiti Coast of New Zealand was cleared of teaching racist theories. An independent investigation concluded that while there were no racist elements in the curriculum, the school needed to make changes in the "areas of governance, management and teaching to ensure parents\' complaints were dealt with appropriately in the future...[and that]...the school must continue regular communication with the school community regarding the ongoing work being undertaken to address the issues raised and noted that the board has proactively sought support to do this."\n\nRacist attitudes and behaviour have been reported in particular Waldorf schools, and some teachers have reportedly expressed Steiner\'s view that individuals reincarnate through various races, however, Kevin Avison, senior advisor for the Steiner Waldorf Schools Fellowship in the UK and Ireland, calls the claim of belief in reincarnation through the races "a complete and utter misunderstanding" of Steiner\'s teachings.\n\nSee also\n\nReferences\n\nFurther reading \n\n Clouder, Christopher (ed.). Education: An Introductory Reader. Sophia Books, 2004 ("a collection of relevant works by Steiner on education").\n Steiner, Rudolf. "The Education of the Child, and early Lectures on Education" in Foundations of Waldorf Education, Anthroposophic Press, 1996 (includes Steiner\'s first descriptions of child development, originally published as a small booklet).\n Steiner, Rudolf. The Foundations of Human Experience (also known as The Study of Man). Anthroposophic Press, 1996 (these fundamental lectures on education were given to the teachers just before the opening of the first Waldorf school in Stuttgart in 1919).\n\nNote: All of Steiner\'s lectures on Waldorf education are available in PDF form at this research site.\n\nExternal links\n\n Online Waldorf Library\n Education Section at the Rudolf Steiner Archive, An Online Library\n Interactive map of Waldorf kindergartens, schools and teacher training colleges worldwide\n\nRegional associations of schools\n Association of Waldorf Schools of North America\n Steiner Waldorf Schools Fellowship (UK)\n Steiner Education Australia\n\n \n1919 introductions\nAnthroposophy\nPedagogy\nPhilosophy of education\nProgressive education\nSchool types\nDevelopmental stage theories
"""

In [120]:
model=granite_retriever
# Encode the full document (no chunking needed for <8192 tokens)
doc_embedding = model.encode(long_document, show_progress_bar=True)

# Compare with shorter query
query = "What strengths of Waldorf education could state schools benefit from?"
query_embedding = model.encode(query)
similarity = util.cos_sim(query_embedding, doc_embedding)
print(f"Relevance score for '{query}: {similarity.item():.4f}")

query = "How is Stanford university education compared with Cornell's?"
query_embedding = model.encode(query)
similarity = util.cos_sim(query_embedding, doc_embedding)
print(f"Relevance score for '{query}': {similarity.item():.4f}")

Batches: 100%|██████████| 1/1 [00:00<00:00,  9.32it/s]

Relevance score for 'What strengths of Waldorf education could state schools benefit from?: 0.9093
Relevance score for 'How is Stanford university education compared with Cornell's?': 0.7575


## Code Search Example

This example demonstrates how to perform semantic search on code snippets using natural language queries. It showcases the Granite R2 embedding model's ability to understand both code syntax and semantic meaning, enabling developers to find relevant code implementations through conversational queries.



In [121]:
# Code snippets corpus
code_snippets = [
    """
    def binary_search(arr, target):
        left, right = 0, len(arr) - 1
        while left <= right:
            mid = (left + right) // 2
            if arr[mid] == target:
                return mid
            elif arr[mid] < target:
                left = mid + 1
            else:
                right = mid - 1
        return -1
    """,
    """
    def quicksort(arr):
        if len(arr) <= 1:
            return arr
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quicksort(left) + middle + quicksort(right)
    """,
    """
    class LinkedList:
        def __init__(self):
            self.head = None

        def append(self, data):
            if not self.head:
                self.head = Node(data)
                return
            current = self.head
            while current.next:
                current = current.next
            current.next = Node(data)
    """
]

In [122]:
def code_search(query):
    query_embedding = model.encode(query)

    # Find most relevant code
    similarities = util.cos_sim(query_embedding, code_embeddings)[0]
    best_match = np.argmax(similarities)

    print(f"Most relevant code snippet for the question '{query}':\n{code_snippets[best_match]}")

# Encode code
code_embeddings = model.encode(code_snippets)

# Natural language query
code_search("How do I implement a search algorithm?")
code_search("How to sort in python")

Most relevant code snippet for the question 'How do I implement a search algorithm?':

    def binary_search(arr, target):
        left, right = 0, len(arr) - 1
        while left <= right:
            mid = (left + right) // 2
            if arr[mid] == target:
                return mid
            elif arr[mid] < target:
                left = mid + 1
            else:
                right = mid - 1
        return -1
    
Most relevant code snippet for the question 'How to sort in python':

    def quicksort(arr):
        if len(arr) <= 1:
            return arr
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quicksort(left) + middle + quicksort(right)
    


Table Retrieval and Structured Data Search

Handle structured data with ease using Granite R2 embeddings for semantic table search and information extraction.


In [123]:
# Tables in markdown format
tables = [
    """
    | Product | Q1 Revenue | Q2 Revenue |
    |---------|-----------|-----------|
    | Product A | $500K | $650K |
    | Product B | $300K | $420K |
    """,
    """
    | Employee | Department | Salary |
    |----------|-----------|--------|
    | John Doe | Engineering | $120K |
    | Jane Smith | Marketing | $95K |
    """,
    """
    | Country | Population | GDP |
    |---------|-----------|-----|
    | USA | 331M | $21T |
    | China | 1.4B | $14T |
    """
]

In [124]:
# Encode tables
table_embeddings = model.encode(tables)

# Query for specific information
query = "What was the performance for our products?"
query_embedding = model.encode(query)

similarities = util.cos_sim(query_embedding, table_embeddings)[0]
best_table = np.argmax(similarities)
print(f"Most relevant table:\n{tables[best_table]}")

Most relevant table:

    | Product | Q1 Revenue | Q2 Revenue |
    |---------|-----------|-----------|
    | Product A | $500K | $650K |
    | Product B | $300K | $420K |
    


# Batch Processing for Production Environments

Optimized pipeline for encoding large document collections using Granite R2 embeddings with efficient memory management, GPU acceleration, and persistent storage capabilities for production-scale semantic search deployments.

For additional information, we added throughput and latency information.


In [125]:
import time
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer
from IPython.display import display

In [126]:
corpus = [
    {
        'id': 1,
        'text': 'Sarah discovered an old telescope in her grandmother\'s attic. Every night she would gaze at the stars, dreaming of distant galaxies. One evening, she spotted a comet that hadn\'t been seen in 75 years.'
    },
    {
        'id': 2,
        'text': 'The small bakery on Maple Street had been run by the Chen family for three generations. Their famous sourdough recipe was a closely guarded secret, passed down from grandmother to mother to daughter.'
    },
    {
        'id': 3,
        'text': 'Marcus trained for months to complete his first marathon. On race day, he struggled at mile 20 but remembered his coach\'s advice. He crossed the finish line in 4 hours and 15 minutes, tears of joy streaming down his face.'
    },
    {
        'id': 4,
        'text': 'The ancient library contained books dating back 500 years. Elena, the new librarian, found a hidden manuscript behind a loose brick. It was a collection of folk tales that had been lost for centuries.'
    },
    {
        'id': 5,
        'text': 'Jake adopted a golden retriever named Max from the local animal shelter. Max had been abandoned but quickly became Jake\'s best friend. They went on daily walks through the park, rain or shine.'
    },
    {
        'id': 6,
        'text': 'The jazz band rehearsed every Tuesday night in an old warehouse. Their drummer, Lisa, had played for 15 years and could make the rhythms dance. They were preparing for their biggest performance at the city festival.'
    },
    {
        'id': 7,
        'text': 'Tom planted a vegetable garden in his backyard last spring. He grew tomatoes, carrots, and bell peppers. By August, he had harvested enough vegetables to share with all his neighbors.'
    },
    {
        'id': 8,
        'text': 'The lighthouse keeper had worked alone for 20 years on the rocky shore. Every evening at sunset, he would climb the 147 steps to light the beacon that guided ships safely to harbor.'
    },
    {
        'id': 9,
        'text': 'Nina learned to paint watercolors during the pandemic. She started with simple landscapes but soon developed her own unique style. Her first art show featured 30 paintings and sold out in two hours.'
    },
    {
        'id': 10,
        'text': 'The old train station was being converted into a community center. Volunteers worked weekends to restore the original wooden benches and vintage ticket booths. It would open to the public in December.'
    }
]

# 4 questions answered by specific stories
questions = [
    {
        'id': 1,
        'question': 'Who found a hidden manuscript in an ancient library?',
        'answer': 'Elena, the new librarian',
        'story_id': 4
    },
    {
        'id': 2,
        'question': 'How long did it take Marcus to complete his first marathon?',
        'answer': '4 hours and 15 minutes',
        'story_id': 3
    },
    {
        'id': 3,
        'question': 'How many steps did the lighthouse keeper climb every evening?',
        'answer': '147 steps',
        'story_id': 8
    },
    {
        'id': 4,
        'question': 'What was the name of the dog that Jake adopted from the animal shelter?',
        'answer': 'Max, a golden retriever',
        'story_id': 5
    }
]


In [147]:
# Run the search and display the results
def do_search(embeddings, query, top_k=10, retriever=model, reranker=None):
    qtext = query['question']
    hits = search(documents, embeddings, qtext, top_k=10, retriever=model, reranker=reranker)
    print(f"The search results for question '{qtext}' are:", flush=True)
    hits_df = pd.DataFrame(hits)
    hits_df['document'] = hits_df['corpus_id'].apply(lambda x: documents[x]['text'])
    hits_df['id'] = hits_df['corpus_id'].apply(lambda x: documents[x]['id'])
    hits_df['answer'] = [query['answer'] for _ in range(len(hits_df))]
    hits_df['gold_id'] = [query['story_id'] for _ in range(len(hits_df))]
    hits_df = hits_df[['id', 'gold_id', 'score', 'document', 'answer']]  # Reorder columns
    #print(f"\nResults:")
    display(hits_df[:top_k])


In [148]:
device = ('cuda' if torch.cuda.is_available() else
          'mps' if torch.backends.mps.is_available() else
          'cpu'
          )

if granite_retriever is not None:
    model = granite_retriever
    granite_retriever.to(device)
else:
    model = SentenceTransformer('ibm-granite/granite-embedding-english-r2',
                                device=device,
                                model_kwargs=model_kwargs
                                )


# Large batch of documents
#documents = corpus * 2000
documents = corpus * 2000

# Document encoding with speed measurement
print(f"Encoding {len(documents)} documents...")
start_time = time.time()

batch_size = 64
all_embeddings = model.encode_document(
    [d['text'] for d in documents],
    batch_size=batch_size,
    show_progress_bar=True,
    convert_to_tensor=True,
    normalize_embeddings=True  # For cosine similarity
)

encoding_time = time.time() - start_time
docs_per_second = len(documents) / encoding_time
# print(f"Documents encoding completed in {encoding_time:.2f}s")
# print(f"Speed: {docs_per_second:.1f} documents/second")

# Save embeddings for later use
torch.save(all_embeddings, 'document_embeddings.pt')
# Load and search later
embeddings = torch.load('document_embeddings.pt')

search_start_time = time.time()

print(f"Running with reranker:")
for q in questions:
    do_search(embeddings, q, top_k=3, retriever=model, reranker=granite_reranker)

# Compute timing
search_time_w_reranker = time.time() - search_start_time

search_start_time = time.time()

print(f"Running without reranker:")
for q in questions:
    do_search(embeddings, q, top_k=3, retriever=model, reranker=None)

# Compute timing
search_time_wo_reranker = time.time() - search_start_time

# Convert hits to pandas DataFrame for better display

print(f"\nPerformance Summary:")
print(f"- Document encoding:           {docs_per_second:.1f} docs/sec ({encoding_time:.2f}s total)")
print(f"- Search latency w/o reranker: {search_time_wo_reranker * 1000 / len(questions):.2f}ms per query")
print(f"- Search latency w/  reranker: {search_time_w_reranker  * 1000 / len(questions):.2f}ms per query")


Encoding 20000 documents...


Batches: 100%|██████████| 313/313 [00:10<00:00, 31.23it/s]


Running with reranker:
The search results for question 'Who found a hidden manuscript in an ancient library?' are:


,id,gold_id,score,document,answer
0,4,4,1.0,The ancient library contained books dating bac...,"Elena, the new librarian"
1,4,4,1.0,The ancient library contained books dating bac...,"Elena, the new librarian"
2,4,4,1.0,The ancient library contained books dating bac...,"Elena, the new librarian"


The search results for question 'How long did it take Marcus to complete his first marathon?' are:


...

The search results for question 'How many steps did the lighthouse keeper climb every evening?' are:


,id,gold_id,score,document,answer
0,8,8,0.933594,The lighthouse keeper had worked alone for 20 ...,147 steps
1,8,8,0.933594,The lighthouse keeper had worked alone for 20 ...,147 steps
2,8,8,0.933594,The lighthouse keeper had worked alone for 20 ...,147 steps


The search results for question 'What was the name of the dog that Jake adopted from the animal shelter?' are:


,id,gold_id,score,document,answer
0,5,5,0.925781,Jake adopted a golden retriever named Max from...,"Max, a golden retriever"
1,5,5,0.925781,Jake adopted a golden retriever named Max from...,"Max, a golden retriever"
2,5,5,0.925781,Jake adopted a golden retriever named Max from...,"Max, a golden retriever"



Performance Summary:
- Document encoding:           1889.2 docs/sec (10.59s total)
- Search latency w/o reranker: 44.63ms per query
- Search latency w/  reranker: 82.36ms per query


## Multi-Turn Conversational Search

This section demonstrates how **Granite R2 embeddings excel at multi-turn conversational search**, where queries build upon previous context and require understanding of conversational flow and implicit references.

### Key Capabilities Demonstrated:

- **Contextual Understanding**: Process queries that reference previous conversation turns
- **Implicit Reference Resolution**: Handle pronouns and contextual references across turns  
- **Conversational Memory**: Maintain semantic coherence across extended dialogues
- **Turn-by-Turn Retrieval**: Search within individual conversation segments or entire dialogues

### Multi-Turn Search Features:

The following examples showcase how to search through conversational data where:
- **Context Matters**: Questions may reference earlier parts of the conversation
- **Implicit Queries**: Users ask follow-up questions without repeating full context
- **Conversational Flow**: Understanding requires knowledge of the dialogue progression
- **Cross-Turn Dependencies**: Answers depend on information from multiple conversation turns

This capability makes Granite R2 particularly effective for customer support systems, chatbot knowledge retrieval, and any application requiring search across conversational or dialogue data.


In [22]:
conversation_corpus = [
    {'_id': '3',
     'text': "I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their job - they're building software. Perhaps educational systems in the U.S. (or their students) should worry a little about getting marketable skills in exchange for their massive investment in education, rather than getting out with thousands in student debt and then complaining that they aren't qualified to do anything.",
     },
    {'_id': '31',
    'text': "So nothing preventing false ratings besides additional scrutiny from the market/investors, but there are some newer controls in place to prevent institutions from using them. Under the DFA banks can no longer solely rely on credit ratings as due diligence to buy a financial instrument, so that's a plus. The intent being that if financial institutions do their own leg work then *maybe* they'll figure out that a certain CDO is garbage or not.  Edit: lead in",
     },
    {'_id': '56',
    'text': "You can never use a health FSA for individual health insurance premiums.  Moreover, FSA plan sponsors can limit what they are will to reimburse.  While you can't use a health FSA for premiums, you could previously use a 125 cafeteria plan to pay premiums, but it had to be a separate election from the health FSA. However, under N. 2013-54, even using a cafeteria plan to pay for indivdiual premiums is effectively prohibited.",
     },
    {'_id': '59',
    'text': 'Samsung created the LCD and other flat screen technology like OLED. a few years ago every flat screen came from Samsung factories and were reshelled. I think the 21 Hanns screen I am looking at now is Samsung and it is only a couple of years old. Samsung seem to be a good company.',
     },
    {'_id': '63',
    'text': 'Here are the SEC requirements: The federal securities laws define the term accredited investor in   Rule 501 of Regulation D as: a bank, insurance company, registered investment company, business development company, or small business investment company; an employee benefit plan, within the meaning of the Employee Retirement Income Security Act, if a bank, insurance company, or   registered investment adviser makes the investment decisions, or if   the plan has total assets in excess of $5 million; a charitable organization, corporation, or partnership with assets exceeding $5 million; a director, executive officer, or general partner of the company selling the securities; a business in which all the equity owners are accredited investors; a natural person who has individual net worth, or joint net worth with the person’s spouse, that exceeds $1 million at the time of the   purchase, excluding the value of the primary residence of such person; a natural person with income exceeding $200,000 in each of the two most recent years or joint income with a spouse exceeding $300,000 for   those years and a reasonable expectation of the same income level in   the current year; or a trust with assets in excess of $5 million, not formed to acquire the securities offered, whose purchases a sophisticated person makes. No citizenship/residency requirements.',
     },
    {'_id': '100',
    'text': '"Only relevant to those with fantasy economy teams. Seriously, Rand\'s fictional works never translate well into reality because, no matter how hard people try, that ""fiction"" element just can\'t be ignored.  Test it yourself: Strip John Galt and his followers of everything they have which was created by or within the ""society"" they so revile, drop them in the desert -- and they\'ll all be dead of exposure and starvation in less than two weeks because they will be naked, without tools and without food.  The only reason the libertarians get away with pushing their tripe as a rational philosophy is because no one will point out what it is wrong with their thinking. Why? Well, for most of my lifetime, their ""philosophy"" was considered nuttery in line with the John Birchers and so why bother. It\'s only with the ascendency of these billionaire-funded politicians that this crap thinking has become acceptable, and even then, only to them."',
     },
    {'_id': '108',
    'text': "Futures contracts are a member of a larger class of financial assets called derivatives. Derivatives are called such because their payoffs depend on the price of other assets (financial or real). Other kinds of derivatives are call options, put options. Fixed income assets that mimic the behavior of derivatives are callable bonds, puttable bonds etc.  A futures contract is a contract that specifies the following: Just like with any other contract, there are two parties involved. One party commits to delivering the underlying asset to the other party on expiration date in exchange for the futures price. The other party commits to paying the futures price in exchange for the asset. There is no price that any of the two parties pay upfront to engage in the contract. The language used is so that the agent committing to receiving the delivery of the underlying asset is said to have bought the contract. The agent that commits to make the delivery is said to have sold the contract.  So answer your question, buying on June 1 a futures contract at the futures price of $100, with a maturity date on August 1 means you commit to paying $100 for the underlying asset on August 1. You don't have to pay anything upfront. Futures price is simply what the contract prescribes the underlying asset will exchange hands for.",
     },
    {'_id': '125',
    'text': 'This month when you join Scentsy you get a free defuser with your kit!   This has never been done before.  You also get spring / summer and Fall / Winter testers plus all your kit items!    Be your own boss!  You choose what hours you work, when and where you work them.   Join my Scentsy family today!  [Amanda C. Robar Scentsy Business ](http://www.amandacrobar.scentsy.ca)',
     },
    {'_id': '132',
    'text': 'Whenever you pay or withdraw some fund from your account, paypal takes approx 3% of the current currency value along with the fees. i.e. If you are paying/withdraw 100 unit of US Dollars to British pounds and if the current convertion rate is 1$=0.82GBP, then consider reducing 3% of the actual currency rate. So, the approximate magnitude will be 0.82*97% (100-3=97) = 0.7954. So, 1$=0.7954GBP. This formula will not give you 100% accurate value but will help of course. Captain',
     },
    {'_id': '138',
    'text': 'So you asked him in 2010 how he was gong to compete with DVD rental distributors like Netflix (which is what Netflix primarily was at the time) and Lovefilm and you were surprised that he was he said they were going to continue to compete as a DVD rental distributor just like the mentioned competitors?',
     }
    ]

In [23]:
retriever = granite_retriever


In [60]:
# Concatenate conversation context
text_corpus = [s['text'] for s in conversation_corpus]
conv_embeddings = retriever.encode_document(text_corpus, convert_to_tensor=True)

In [61]:
def run_query(query, corpus_embeddings=conv_embeddings, retriever=retriever, reranker=None, top_k=3):
    results = search(corpus=text_corpus,
                     corpus_embeddings=corpus_embeddings,
                     query=query, retriever=retriever,
                     reranker=reranker,
                     top_k=top_k)
    print(f"Results for the query: '{query}'")
    for hit in results:
        print(f"\tScore: {hit['score']:.4f} | {conversation_corpus[hit['corpus_id']]['text'][:140]} [...]")
    print()

    return results

In [62]:
# Search in your knowledge base
queries = [
    "Who invented the OLED screen?",
    "How real is Rand's writing?",
    "What's a futures contract?"
    ]
for query in queries:
    run_query(query)


TypeError: search() got an unexpected keyword argument 'corpus'

### Putting it all together in one class

This section introduces the `SemanticSearchManager` class, which consolidates all the previously demonstrated functionality into a comprehensive, production-ready semantic search system. The class provides a unified interface for:

- **Model Management**: Automatic loading of both retriever and reranker models with GPU optimization
- **Corpus Processing**: Efficient batch encoding and storage of document embeddings
- **Flexible Search**: Configurable retrieval with optional reranking for optimal accuracy
- **Performance Optimization**: Hardware-aware configuration and memory-efficient batch processing
- **Easy Integration**: Simple API that abstracts away the complexity of the two-stage retrieval pipeline

The class encapsulates best practices for semantic search deployment, including proper error handling, device management, and result formatting, making it straightforward to integrate advanced embedding-based search capabilities into any application.


In [29]:
from typing import List, Dict, Optional, Union
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch
import numpy as np

DEFAULT_RETRIEVER_MODEL = 'ibm-granite/granite-embedding-english-r2'
DEFAULT_RERANKER_MODEL = 'ibm-granite/granite-embedding-reranker-english-r2'
DEFAULT_EMBEDDING_DIM = 768
DEFAULT_TOP_K = 20
DEFAULT_TOP_RESULTS = 5


class SemanticSearchManager:
    """
    A comprehensive semantic search system that combines retrieval and reranking
    for enhanced search accuracy and performance.
    """

    def __init__(self,
                 retriever_model: str = DEFAULT_RETRIEVER_MODEL,
                 reranker_model: str = DEFAULT_RERANKER_MODEL,
                 use_gpu: bool = True):
        """
        Initialize the semantic search manager with retriever and reranker models.

        Args:
            retriever_model: Name of the SentenceTransformer model for retrieval
            reranker_model: Name of the CrossEncoder model for reranking
            use_gpu: Whether to use GPU acceleration if available
        """
        self.retriever_model_name = retriever_model
        self.reranker_model_name = reranker_model
        self.corpus_embeddings = None
        self.corpus = None

        # Configure model kwargs based on hardware availability
        self.model_kwargs = self._configure_model_kwargs(use_gpu)

        # Initialize models
        self.retriever = self._load_retriever()
        self.reranker = self._load_reranker()

    def __del__(self):
        del self.retriever
        del self.reranker

    def _configure_model_kwargs(self, use_gpu: bool) -> Dict:
        """Configure model parameters based on hardware availability."""
        if use_gpu and torch.cuda.is_available():
            return {
                "torch_dtype": torch.bfloat16,
                'attn_implementation': 'flash_attention_2'
            }
        return {}

    def _load_retriever(self) -> SentenceTransformer:
        """Load and configure the retriever model."""
        return SentenceTransformer(self.retriever_model_name,
                                   model_kwargs=self.model_kwargs,
                                   device=SemanticSearchManager._get_device())

    def _load_reranker(self) -> Optional[CrossEncoder]:
        """Load and configure the reranker model."""
        try:
            return CrossEncoder(self.reranker_model_name,
                                model_kwargs=self.model_kwargs,
                                device=SemanticSearchManager._get_device())
        except Exception as e:
            print(f"Warning: Could not load reranker model: {e}")
            return None

    @staticmethod
    def _get_device():
        device = ('cuda' if torch.cuda.is_available() else
                  "mps" if torch.backends.mps.is_available() else
                  'cpu'
                  )
        return device

    def encode_corpus(self, corpus: List[str], show_progress: bool = True):
        """
        Encode the document corpus for semantic search.

        Args:
            corpus: List of documents to encode
            show_progress: Whether to show encoding progress
        """
        self.corpus = [self.get_text(s) for s in corpus]
        self.corpus_embeddings = self.retriever.encode_document(
            self.corpus,
            convert_to_tensor=True,
            show_progress_bar=show_progress
        )
        return self.corpus_embeddings

    def encode_query(self, query: str) -> torch.Tensor:
        """Encode a single query for semantic search."""
        return self.retriever.encode_query(query, convert_to_tensor=True)

    def search(self,
               query: str,
               top_k: int = DEFAULT_TOP_K,
               top_results: int = DEFAULT_TOP_RESULTS,
               use_reranker: bool = True,
               print_results: bool=False,
               max_size: int=-1) -> List[Dict]:
        """
        Perform semantic search with optional reranking.

        Args:
            query: Search query string
            top_k: Number of candidates to retrieve initially
            top_results: Final number of results to return
            use_reranker: Whether to use reranker for final ranking

        Returns:
            List of search results with scores and content
        """
        if self.corpus_embeddings is None:
            raise ValueError("Corpus must be encoded first using encode_corpus()")

        # Step 1: Retrieve top-k candidates
        query_embedding = self.encode_query(query)
        hits = util.semantic_search(query_embedding, self.corpus_embeddings, top_k=top_k)[0]

        # Step 2: Optionally rerank with cross-encoder
        if use_reranker and self.reranker is not None:
            hits = self._rerank_results(query, hits)

        if print_results:
            self.print_results(query, hits[:top_results], max_size=max_size)

        return hits[:top_results]

    def _rerank_results(self, query: str, hits: List[Dict]) -> List[Dict]:
        """Apply reranking to search results."""
        cross_input = [[query, self.corpus[hit['corpus_id']]] for hit in hits]
        cross_scores = self.reranker.predict(cross_input)

        # Update hits with reranking scores
        for idx, score in enumerate(cross_scores):
            hits[idx]['rerank_score'] = score

        # Sort by reranker scores
        return sorted(hits, key=lambda x: x['rerank_score'], reverse=True)

    def get_text(self, item):
        if isinstance(item, str):
            return item
        elif isinstance(item, dict) and 'text' in item:
            return item['text']

    def print_results(self, query: str, results, top_results: int = DEFAULT_TOP_RESULTS,
                      max_size=-1):

        print(f"Results for the query: '{query}'")
        for hit in results:
            txt = self.get_text(self.corpus[hit['corpus_id']])
            if max_size > 0 and len(txt) > max_size:
                txt = f"{txt[:max_size]} [...]"
            print(f"\tScore: {hit['score']:.4f} | {txt}")
        print()

    def simple_similarity_search(self, query: str, documents: List[str]) -> Dict:
        """
        Perform simple cosine similarity search without corpus pre-encoding.
        Useful for one-time searches on small document sets.
        """
        query_embedding = self.encode_query(query)
        _ = self.encode_corpus(documents)

        similarities = util.cos_sim(query_embedding, self.corpus_embeddings)
        best_idx = torch.argmax(similarities)

        return {
            'similarities': similarities,
            'best_match_index': int(best_idx),
            'best_match': documents[best_idx],
            'best_score': float(similarities[0][best_idx])
        }


In [30]:
# Example: Simple similarity search
def run_experiment(manager):
    documents = [
        "Granite models are designed for enterprise applications",
        "Information retrieval systems need fast and accurate embeddings",
        "Machine learning models can process natural language"
    ]

    query = "What's the purpose of the granite models?"
    print("=" * 80, "\n", f"Running with {manager.retriever_model_name}", "=" * 80)
    manager.encode_corpus(documents)
    manager.search(query, top_k=5, use_reranker=False, print_results=True, max_size=-1)

    manager.encode_corpus(conversation_corpus)
    query = "Who invented the OLED screen?"
    query1 = "How real is Rand's writing?"
    manager.search(query, top_k=5, print_results=True, max_size=140, use_reranker=False)
    manager.search(query1, top_k=5, print_results=True, max_size=140, use_reranker=False)

search_manager = SemanticSearchManager()
run_experiment(search_manager)


 Running with ibm-granite/granite-embedding-english-r2 ================================================================================


Batches: 100%|██████████| 1/1 [00:00<00:00, 41.76it/s]


Results for the query: 'What's the purpose of the granite models?'
	Score: 0.8750 | Granite models are designed for enterprise applications
	Score: 0.7344 | Machine learning models can process natural language
	Score: 0.7188 | Information retrieval systems need fast and accurate embeddings



Batches: 100%|██████████| 1/1 [00:00<00:00, 37.41it/s]

Results for the query: 'Who invented the OLED screen?'
	Score: 0.8672 | Samsung created the LCD and other flat screen technology like OLED. a few years ago every flat screen came from Samsung factories and were r [...]
	Score: 0.6875 | So you asked him in 2010 how he was gong to compete with DVD rental distributors like Netflix (which is what Netflix primarily was at the ti [...]
	Score: 0.6719 | This month when you join Scentsy you get a free defuser with your kit!   This has never been done before.  You also get spring / summer and  [...]
	Score: 0.6523 | I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their  [...]
	Score: 0.6523 | "Only relevant to those with fantasy economy teams. Seriously, Rand's fictional works never translate well into reality because, no matter h [...]

Results for the query: 'How real is Rand's writing?'
	Score: 0.8672 | "Only relevant to those with fantasy economy teams. Se

 Running with google/embeddinggemma-300m ================================================================================


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.18it/s]


Results for the query: 'What's the purpose of the granite models?'
	Score: 0.6250 | Granite models are designed for enterprise applications
	Score: 0.4375 | Machine learning models can process natural language
	Score: 0.3770 | Information retrieval systems need fast and accurate embeddings



Batches: 100%|██████████| 1/1 [00:00<00:00, 20.69it/s]


Results for the query: 'Who invented the OLED screen?'
	Score: 0.4766 | Samsung created the LCD and other flat screen technology like OLED. a few years ago every flat screen came from Samsung factories and were r [...]
	Score: 0.3359 | So you asked him in 2010 how he was gong to compete with DVD rental distributors like Netflix (which is what Netflix primarily was at the ti [...]
	Score: 0.3320 | I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their  [...]
	Score: 0.3301 | This month when you join Scentsy you get a free defuser with your kit!   This has never been done before.  You also get spring / summer and  [...]
	Score: 0.2910 | So nothing preventing false ratings besides additional scrutiny from the market/investors, but there are some newer controls in place to pre [...]

Results for the query: 'How real is Rand's writing?'
	Score: 0.3984 | "Only relevant to those with fantasy economy teams. Se